In [1]:
!pip install pulp

In [2]:
import pandas as pd
import numpy as np
import pulp
import os
os.getcwd()
os.chdir("..")

# Process data

In [3]:
raw_workmode = pd.read_excel('InputData.xlsx', sheet_name="Mode")
raw_demand = pd.read_excel('InputData.xlsx', sheet_name="Demand")
workmode = {(row['Mode'].item(), row['Shift'].item()): row['Capa'].item() for _, row in raw_workmode.iterrows()}
demand = {(row['Week'].item(), row['Wday'].item()): row['Demand'].item() for _, row in raw_demand.iterrows()}

In [4]:
WeekIndex = [1,2,3,4]
WeekDayIndex = [1,2,3,4,5,6,7]
ShiftIndex = [1,2,3]

In [5]:
numS = 22
numC = 12
FC = {(i, j): row['Hours'].item()*8.3 for i in [1, 2] for j in ShiftIndex for _, row in raw_workmode.iterrows()}


# Algorithm

In [6]:
workmode[1, 1]

7182

In [7]:
demand[1, 1]

7857

In [8]:
m_conso_sample = {1: {1: {1: 1,
   2: 0,
   3: 1,
   4: 0,
   5: 0,
   6: 1,
   7: 1}}}

m_conso_sample[1][1][1]

1

In [9]:
sum(m_conso_sample[1][1][1]*workmode[1, sh] for sh in ShiftIndex)

27896

In [10]:
# Create the model
model = pulp.LpProblem("3PL_Workforce_Allocation", pulp.LpMinimize)

# Define binary decision variables x (3D) and y (2D)
m_conso = pulp.LpVariable.dicts("m_conso", (ShiftIndex, WeekIndex, WeekDayIndex), cat='Binary')
m_sepa = pulp.LpVariable.dicts("m_sepa", (ShiftIndex, WeekIndex, WeekDayIndex), cat='Binary')

# Objective function: 
model += pulp.lpSum(numC*FC[1, sh] * m_conso[sh][w][d] for sh in ShiftIndex for w in WeekIndex for d in WeekDayIndex) + \
         pulp.lpSum(numS*FC[2, sh] * m_sepa[sh][w][d] for sh in ShiftIndex for w in WeekIndex for d in WeekDayIndex)

# Constraint
## Can't work 3 consecutive days
for w in WeekIndex:
    for d in range(1, len(WeekDayIndex) - 1):
        model += pulp.lpSum(m_conso[3][w][d+t] for t in [0,1,2]) <= 2
for w in WeekIndex:
    for d in range(1, len(WeekDayIndex) - 1):
        model += pulp.lpSum(m_sepa[3][w][d+t] for t in [0,1,2]) <= 2

## Can't use the same mode within a week
# for w in WeekIndex:
#     model += pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) <= 7
# for w in WeekIndex:
#     model += pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) <= 7
# for w in WeekIndex:
#     model += pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) + pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) >= 7

# Satisfy demand
for w in WeekIndex:
    for d in WeekDayIndex:
        model += pulp.lpSum(m_sepa[sh][w][d]*workmode[1, sh] for sh in ShiftIndex) + pulp.lpSum(m_conso[sh][w][d]*workmode[2, sh] for sh in ShiftIndex) >= demand[w, d]

# One shift only
for w in WeekIndex:
    for d in WeekDayIndex:
        model += pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex) <= 1
for w in WeekIndex:
    for d in WeekDayIndex:
        model += pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex) <= 1
        

# Solve the problem using COIN_CMD without the tol argument
solver = pulp.PULP_CBC_CMD(gapRel=0.00001)  # Built-in solver
model.solve(solver)

# Collect the results into a list of dictionaries for x_ijk (3D)
results_conso = []
for sh in ShiftIndex:
    for w in WeekIndex:
        for d in WeekDayIndex:
            results_conso.append({
                'shift': sh,
                'week': w,
                'wday': d,
                'choose': m_conso[sh][w][d].varValue
            })

# Collect the results into a list of dictionaries for y_ij (2D)
results_sepa = []
for sh in ShiftIndex:
    for w in WeekIndex:
        for d in WeekDayIndex:
            results_sepa.append({
                'shift': sh,
                'week': w,
                'wday': d,
                'choose': m_sepa[sh][w][d].varValue
            })

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lamjackie/miniconda3/envs/ych-wf-allocation/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/e6279f6f54f14869991d775395b4c663-pulp.mps -ratio 1e-05 -timeMode elapsed -branch -printingOptions all -solution /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/e6279f6f54f14869991d775395b4c663-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 129 COLUMNS
At line 1090 RHS
At line 1215 BOUNDS
At line 1384 ENDATA
Problem MODEL has 124 rows, 168 columns and 456 elements
Coin0008I MODEL read with 0 errors
ratioGap was changed from 0 to 1e-05
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18927.3 - 0.00 seconds
Cgl0003I 0 fixed, 21 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 124 rows, 147 columns (147 integer (147 of which binary)) and 414 elem

In [11]:
df_results_conso=pd.DataFrame(results_conso)
df_results_conso['mode'] = "c"
df_results_sepa = pd.DataFrame(results_sepa)
df_results_sepa['mode'] = "s"
result = pd.concat([df_results_sepa, df_results_conso])
result_pivot=pd.DataFrame(result.pivot_table(index=["mode", "shift"], columns=["week", "wday"], values=["choose"], aggfunc="sum").reset_index(drop=False))

In [12]:
with pd.ExcelWriter("result.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    result_pivot.to_excel(writer, sheet_name="Pivot")
    result.to_excel(writer, sheet_name="Results", index=False)
    raw_workmode.to_excel(writer, sheet_name="Workforce", index=False)
    raw_demand.to_excel(writer, sheet_name="Demand", index=False)